### 직접 작성 (GPT2LMHeadModel)

In [ ]:
!pip install pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.3/802.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 14.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [ ]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.core import LightningModule
from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re, os
from tqdm import tqdm

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TORCH_USE_CUDA_DSA"] = '1'

In [ ]:
# 하이퍼 파라메터 설정
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = "</s>"
EOS = "</s>"
MASK = "<unused0>"
SENT = "<unused1>"
PAD = "<pad>"
EPOCHS = 5
Sneg = -1e18
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
learning_rate=3e-5

In [ ]:
# 토크나이저 불러오기
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token=BOS,
                                                    eos_token=EOS,
                                                    unk_token='<unk>',
                                                    pad_token=PAD,
                                                    mask_token=MASK)

# 모델 불러오기
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")

# 데이터셋 가져오기
data_url = "https://raw.githubusercontent.com/IDIOcoder/Chat-bot/main/dataset/answer_dataset.csv"
data_set = pd.read_csv(data_url, encoding='utf-8')
data_set.dropna(subset=["Q"], inplace=True)
data_set.dropna(subset=["A"], inplace=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [ ]:
data_set.head(-1)

,Q,A,구분
0,이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너...,그런 일이 있었군요. 실수는 누구에게나 일어날 수 있는 일이에요. 중요한 건 그것을...,대화
1,회사에서 중요한 프로젝트를 혼자 하게 됐는데 솔직히 두렵고 무서워.,"프로젝트를 혼자 맡는 것은 정말 큰 도전일 테지만, 동시에 많은 성장의 기회가 될 ...",대화
2,이번에 힘들게 들어간 첫 직장이거든. 첫 직장이라서 그런지 너무 긴장된다.,첫 직장은 항상 긴장되고 설레는 시간이죠. 새로운 환경에서 적응하는 것은 어려운 일...,대화
3,대회에 나갔는데 예전의 내 기록을 훨씬 갱신했어!,대단하시네요! 축하드립니다! 실력을 계속 향상시키고 성취를 이루는 건 정말 멋진 일...,대화
4,150만 원짜리 노트북을 샀는데 오늘 배송돼서 기분이 너무 좋아!,축하해요! 새로운 노트북으로 더 많은 일들을 해나가실 거예요. 새로운 도구로 무엇을...,대화
...,...,...,...
3830,어떡해! 내일 시험인데 공부를 전혀 안 했어. 어젯밤에 그냥 잠들어버렸어. 시험지 ...,걱정이 많으시겠어요. 지금이라도 조금이라도 공부해보세요. 시험 전에는 긴장하지 않고...,대화
3831,"완전 난리 났어! 이번 시험 공부를 하나도 안 했어. 친구들은 다 공부했을 텐데, ...",불안해하시는 마음 이해해요. 지금이라도 조금이라도 공부해보세요. 시험 전에는 긴장하...,대화
3832,"이번 시험 정말 큰일 났어! 공부를 하려고 했는데, 결국은 그냥 놀기만 했어. 시험...",놀기만 해서 걱정이시겠어요. 지금이라도 조금이라도 공부해보세요. 시험 전에는 긴장하...,대화
3833,헉! 내일 시험인데 공부를 하나도 못 했어. 어제 밤에 잠들어버려서 오늘 하루 종일...,걱정이 많으시겠어요. 지금이라도 조금이라도 공부해보세요. 시험 전에는 긴장하지 않고...,대화


In [ ]:
# 각 문장을 토큰화하고 가장 긴 시퀀스 길이를 찾는 함수
def find_max_len(dataset, tokenizer):
    max_len = 0
    for idx, row in dataset.iterrows():
        q = row['Q']
        a = row['A']

        # 질문과 답변을 토큰화
        q_toked = tokenizer.tokenize(q)
        a_toked = tokenizer.tokenize(a)

        # 토큰화된 질문과 답변의 길이 합산
        total_len = len(q_toked) + len(a_toked)

        # 최대 길이 갱신
        if total_len > max_len:
            max_len = total_len

    return max_len

# 데이터셋의 최대 토큰 길이 찾기
max_len = find_max_len(data_set, tokenizer)
print(f"Max token length in the dataset: {max_len}")

Max token length in the dataset: 121


In [ ]:
# 데이터셋 정의
class ChatbotDataset(Dataset):
  def __init__(self, chats, max_len=40):
    self._data = chats
    self.max_len = max_len
    self.q_token = Q_TKN
    self.a_token = A_TKN
    self.sent_token = SENT
    self.eos = EOS
    self.mask = MASK
    self.tokenizer = tokenizer

  def __len__(self):  # ChatbotDataset의 길이를 리턴.
    return len(self._data)

  def __getitem__(self, idx): # 로드한 ChatbotDataset을 차례차례 DataLoader로 넘겨주는 메서드
    turn = self._data.iloc[idx]
    q = turn["Q"]  # 질문을 가져온다.

    a = turn["A"]  # 답변을 가져온다.

    q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
    q_len = len(q_toked)

    a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
    a_len = len(a_toked)

    # 질문의 길이가 최대길이보다 크면
    if q_len > self.max_len:
      q_toked = q_toked[-(self.max_len // 2) :] # 질문길이를 최대길이의 반으로
      q_len = len(q_toked)

    # 질문의 길이 + 답변의 길이가 최대길이보다 크면
    if q_len + a_len > self.max_len:
      a_len = self.max_len - q_len
      a_toked = a_toked[:a_len]
      a_len = len(a_toked)

    # 답변 labels = [mask, mask, ..., mask, ..., <bos>, ...답변..., <eos>, <pad> ...]
    labels = [self.mask,] * q_len + a_toked[1:]

    # mask = 질문길이 0 + 답변길이 1 + 나머지 0
    mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
    # 답변 labels를 index로 만든다.
    labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
    labels_ids += [self.tokenizer.pad_token_id] * (self.max_len - len(labels_ids))

    #질문 + 답변을 index로 만든다.
    token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
    token_ids += [self.tokenizer.pad_token_id] * (self.max_len - len(token_ids))

    # 질문 + 답변, 마스크, 답변
    return (token_ids, np.array(mask), labels_ids)

In [ ]:
def collate_batch(batch):
  data = [item[0] for item in batch]
  mask = [item[1] for item in batch]
  label = [item[2] for item in batch]
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)

In [ ]:
train_set = ChatbotDataset(data_set, max_len=max_len)
train_dataloader = DataLoader(
    train_set,
    batch_size = 16,
    num_workers=2,  # 윈도우->0 / 리눅스->2
    shuffle=True,
    collate_fn=collate_batch,
)

model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [ ]:
model.to(device)

criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)

for epoch in range(EPOCHS):
  dataloader = tqdm(train_dataloader, desc=f"Epoch {epoch}")
  # train_dataloader에서 배치 단위로 샘플을 가져와 학습.
  for batch_idx, samples in enumerate(dataloader):
    # Gradient 0 으로 초기화
    optimizer.zero_grad()
    token_ids, mask, label = samples

    # 데이터 타입 확인 및 GPU로 이동
    token_ids, mask, label = token_ids.to(device), mask.to(device), label.to(device)

    out = model(token_ids)
    out = out.logits
    # 마스크 3D로 확장 및 출력 필터링
    mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
    mask_out = torch.where(mask_3d == 1, out, Sneg*torch.ones_like(out))

    # 라벨 값이 클래스 개수 범위 내에 있는지 확인
    assert label.max().item() < model.config.vocab_size, "라벨 값이 클래스 개수보다 큽니다."

    loss = criterion(mask_out.transpose(2, 1), label)
    avg_loss = loss.sum() / mask.sum()
    avg_loss.backward()
    optimizer.step()

Epoch 0:   0%|          | 0/240 [00:00<?, ?it/s]<ipython-input-9-b7a6fc7d1fcc>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)
<ipython-input-9-b7a6fc7d1fcc>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
Epoch

In [ ]:
model.to(device)
# 답변 생성 함수
def sample_sequence(model, tokenizer, q, max_len=40, temperature=1.0, top_k=50, top_p=0.9):
    model.eval()
    with torch.no_grad():
        a = ""
        for _ in range(max_len):
            input_ids = torch.LongTensor(
                tokenizer.encode(Q_TKN + q + SENT + A_TKN + a)
            ).unsqueeze(dim=0).to(device)
            pred = model(input_ids)
            logits = pred.logits[:, -1, :] / temperature
            filtered_logits = top_k_top_p_filtering(logits, top_k=top_k, top_p=top_p)
            probabilities = torch.nn.functional.softmax(filtered_logits, dim=-1)
            gen_id = torch.multinomial(probabilities, num_samples=1).item()
            if gen_id >= tokenizer.vocab_size:
                print(f"Invalid token ID: {gen_id}")
                break
            gen = tokenizer.convert_ids_to_tokens([gen_id])[0]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        return a.strip()

# logits를 필터링 하여 top-k 및 top-p 샘플링을 적용
# top_k : 확률이 높은 상위 k개의 토큰 중에서 샘플링.
# top_p : 누적 확률이 p 이하가 되는 상위 토큰들 중에서 샘플링
def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
    # top_k 필터링
    if top_k > 0:
        top_k = min(top_k, logits.size(-1))
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value
    # top_p 필터링
    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(torch.nn.functional.softmax(sorted_logits, dim=-1), dim=-1)
        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0
        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        if indices_to_remove.shape != logits.shape:
                # print(f"indices_to_remove shape mismatch: {indices_to_remove.shape} vs {logits.shape}")
                return logits
        logits[indices_to_remove] = filter_value
    return logits

# 테스트
with torch.no_grad():
    while True:
        q = input("user > ").strip()
        if q == "quit":
            break
        response = sample_sequence(model, tokenizer, q, max_len=max_len, temperature=0.5, top_k=20, top_p=0.9) # temperature : 샘플링 온도로, 낮은 값은 덜 무작위 적이고, 높은 값은 더 무작위적인 결과를 만듬.
        print(f"Chatbot > {response}")

user > 오늘 날씨가 어때?
Chatbot > 오늘 날씨가 어때요? 날씨도 정말 좋은 것 같아요.
user > 안녕?
Chatbot > 안녕하세요! 오늘 하루도 좋은 하루 되세요.
user > 너무 배가 고파
Chatbot > 음식이 너무 맛있으면 기분이 좋으신가요?
user > 맛이있지는 안항
Chatbot > 그게 중요하죠. 혹시 추가적으로 더 맛있는 식사 하세요.
user > 아니 음식이 맛없어
Chatbot > 아니 음식이 맛없다고 느껴질 때는 그 음식을 다시 만들어 먹을 수 있는 기회를 만들어보세요. 그 기회를 잘 활용하면 더 맛있게 드실 수 있을 거예요.
user > 오늘 기분이 어떄?
Chatbot > 오늘 기분이 어때요? 그런 날이 가끔 있어요. 그런 날은 충분히 휴식을 취하며 좋아하는 음악을 듣는 것도 도움이 될 거예요.
user > 오늘 기분이 너무 안좋아
Chatbot > 기분이 안좋아질 때는 자신에게 충분한 휴식을 취하고, 자신에게 긍정적인 생각을 가져보세요.
user > 밥이 너무 맛이없어
Chatbot > 밥의 맛이 없으신다니 정말 아쉽겠어요. 다음에도 맛있게 드세요.
user > 맛이 없다니까
Chatbot > 맛 없으신다니 후회되시겠어요. 다음 번에는 다양한 재료로 맛을 더해보세요.
user > 여행을 가고싶어
Chatbot > 여행을 가고 싶으시군요. 여행의 즐거움과 휴식을 함께 즐기며 건강한 생활을 즐겨보세요.
user > 여행지는 어디가 좋아?=
Chatbot > 여행지는 제주도가 좋아요. 자연경관도 아름답고 맛집도 많아요. 추천해주세요.
user > 여행지는 제주도가 좋아?
Chatbot > 여행지는 여러 곳이 있어요. 자연경관도 아름답고 맛집도 많아요. 한번 가보시면 후회하지 않으실 거예요!
user > 제주도는 어때?
Chatbot > 네, 제주도는 정말 맛있죠! 어떤 음식을 드셨나요?
user > 여행은 어디를 가야할까?
Chatbot > 여행은 주로 제주도가 좋죠. 자연경관도 아름답고 맛집도 많아요.
user > 

In [ ]:
model.to(device)
# 테스트
with torch.no_grad():
  while True:
    q = input("user > ").strip()
    if q == "quit":
      break
    a = ""
    while 1:
      input_ids = torch.LongTensor(
          tokenizer.encode(Q_TKN + q + SENT + A_TKN + a)
      ).unsqueeze(dim=0).to(device)
      pred = model(input_ids)
      pred = pred.logits
      gen_id = torch.argmax(pred, dim=-1).squeeze().cpu().numpy().tolist()[-1]
      gen = tokenizer.convert_ids_to_tokens([gen_id])[0]
      if gen == EOS:
        break
      a += gen.replace("▁", " ")
    print(f"Chatbot > {a.strip()}")

user > 저녁으로 뭘 먹어야 할지 고민이야.
Chatbot > 뭘 먹어야 할지 고민이 많으시군요. 다양한 재료로 디저트를 만들어보는 것도 좋겠네요.
user > 디저트로 저녁을 해결하고 싶지는 않아.
Chatbot > 디저트로 저녁을 해결하고 싶다면 어떤 메뉴를 드시고 싶으세요?
user > 친구들이랑 피크닉 다녀왔어
Chatbot > 피크닉은 친구들과 함께 즐거운 시간을 보내기 좋은 좋은 시간이에요!
user > 친구들이랑 한강으로 놀러 갔다왔어.
Chatbot > 친구들과 함께 한강으로 놀러 가셨군요! 가족과 함께하는 시간은 언제나 소중하고 즐거울 거예요. 안전하고 즐거운 시간 보내시길 바랍니다.
user > quit
